### 1. Library

In [7]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser
import numpy as np
import re

### 2. url connecting

In [9]:
url ='https://lab.donutz.co/krx/products'
resultXML = urlopen(url)
# result = resultXML.read()
# soup = BeautifulSoup(result,'html.parser')
# soup_string = str(soup)
# regex = re.compile(r'(\d{6})')
# code = regex.findall(soup_string)
# code = '000660'
# start_date = '20170101'
# end_date = '20191231'

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>

### 3. Crawling

In [ ]:
url ='https://lab.donutz.co/krx/products/' + code + '?sdate=' + start_date + '&edate=' + end_date
resultXML = urlopen(url)
result = resultXML.read()
soup = BeautifulSoup(result,'html.parser')
soup_string = str(soup)

### 4. 특수문자 제거

In [ ]:
def clean(data):
    text = re.sub('[a-zA-Z-=+#/\?^$@*\"※~&%ㆍ!_』:\\‘|\(\)\[\]\<\>`\'{}…》]', '', data)
    return text
text = re.split('[,]+',clean(soup_string))
information = []
for i in text:
    information.append(i)

### 5. 데이터프레임으로 변환

In [ ]:
data = pd.DataFrame(np.array(information).reshape(-1,15))
data.columns = ['date','open','high','low','close','trading_volume',\
                'score','index','probability','l1','l2','l3','l4','lgap','lrate']

In [ ]:
data.info()

### 6. Data to Float

In [ ]:
data['open'] = data['open'].apply(lambda x : np.float(x))
data['high'] = data['high'].apply(lambda x : np.float(x))
data['low'] = data['low'].apply(lambda x : np.float(x))
data['close'] = data['close'].apply(lambda x : np.float(x))
data['trading_volume'] = data['trading_volume'].apply(lambda x : np.float(x))
data['score'] = data['score'].apply(lambda x : np.float(x))
data['index'] = data['index'].apply(lambda x : np.float(x))
data['probability'] = data['probability'].apply(lambda x : np.float(x))
data['l1'] = data['l1'].apply(lambda x : np.float(x))
data['l2'] = data['l2'].apply(lambda x : np.float(x))
data['l3'] = data['l3'].apply(lambda x : np.float(x))
data['l4'] = data['l4'].apply(lambda x : np.float(x))
data['lgap'] = data['lgap'].apply(lambda x : np.float(x))
data['lrate'] = data['lrate'].apply(lambda x : np.float(x))

### 7. L1 ~ L4 Line 정리

In [ ]:
data['l1'] = data['l1'] - 100000
data['l2'] = data['l2'] - 200000
data['l3'] = data['l3'] - 300000
data['l4'] = data['l4'] - 400000

### 8. Zone Classification

In [ ]:
data['zone'] = 100 * (data['close'].shift(-1) - data['close']) / data['close']

data = data.dropna()

def zone(x) : 
    if x < -5 : 
        return 0
    elif x < -3 : 
        return 1
    elif x < -1 : 
        return 2
    elif x < +1 : 
        return 3
    elif x < +3 : 
        return 4
    elif x < +5 : 
        return 5   
    elif x < +10 : 
        return 6
    else : 
        return 7
    
data['zone'] = data['zone'].apply(lambda x : zone(x))

In [ ]:
del data['date']

### 9. Data

In [ ]:
data

### 10. Zone 구역별 데이터 개수

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(16,8))
sns.countplot(x="zone", data=data)
plt.title("Zone Count Plot")
plt.show()

### 12. Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[data.columns[:-1]], 
                                                    data['zone'], test_size=0.2, random_state=1) 
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 13. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
y_train = y_train.values
y_pred = rf.predict(X_train)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred)

In [ ]:
cm = confusion_matrix(y_train, y_pred)
sum = 0
for i in range(7) : 
    sum += cm[i,i]
print(sum/len(X_train))

In [ ]:
y_test = y_test.values
y_pred = rf.predict(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sum = 0
for i in range(7) : 
    sum += cm[i,i]
print(sum/len(X_test))